In [1]:
# prepend parent dir to import from local pcsp
import sys

sys.path.insert(0, '../')

In [3]:
# % load_ext autoreload
# % autoreload 2
import numpy as np
import pandas as pd
from vflow import Vset, init_args
from vflow.vset import PREV_KEY
from functools import partial
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [4]:
ind = {}


def top_n_features(X, Y, n, i):
    if i not in ind:
        corr = np.abs(np.apply_along_axis(lambda x: np.corrcoef(x, Y[:, i])[0, 1], 0, X))
        ind[i] = np.argsort(corr[~np.isnan(corr)])[::-1][:n]
    return X[:, ind[i]]


def pca(X, n):
    return PCA(n_components=n, copy=True).fit(X).transform(X)


In [5]:
# load data
X = np.asarray(pd.read_csv("../data/fmri/fit_feat.txt", sep=" ", error_bad_lines=False))
Y = np.asarray(pd.read_csv("../data/fmri/resp_dat.txt", sep=" ", error_bad_lines=False))

FileNotFoundError: [Errno 2] No such file or directory: '../data/fmri/fit_feat.txt'

In [5]:
np.random.seed(14)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test),
                                             names=['X_train', 'X_test', 'y_train', 'y_test'])

# split y_train by voxel and extract top 500 correlated features per voxel
voxel_extract_funcs = [partial(lambda x, y, i: (top_n_features(x, y, 20, i), y[:, i]), i=i) for i in range(20)]
voxel_extract_set = Vset(name='voxel_extract', modules=voxel_extract_funcs, output_matching=True)
X_trains, y_trains = voxel_extract_set(X_train, y_train)
X_tests, y_tests = voxel_extract_set(X_test, y_test)

/home/james/miniconda3/envs/pcs-framework/lib/python3.8/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/james/miniconda3/envs/pcs-framework/lib/python3.8/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
# modeling
modeling_set = Vset(name='modeling', modules=[LassoCV()], module_keys=["Lasso"])
modeling_set.fit(X_trains, y_trains)
modeling_set.out

{('X_train', 'voxel_extract_0'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_1'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_2'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_3'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_4'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_5'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_6'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_7'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_8'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train', 'voxel_extract_9'-'voxel_extract', 'y_train', 'Lasso'): LassoCV(),
 ('X_train',
  'voxel_extract_10'-'voxel_extract',
  'y_train',
  'Lasso'): LassoCV(),
 ('X_train',
  'voxel_extract_11'-'voxel_extract',
  'y_train',
  'Lasso'): LassoCV(),
 ('X_train',
  'voxel_extr

In [7]:
preds = modeling_set.predict(X_trains)
for k in preds.keys():
    print(k)

('X_train', 'voxel_extract_0'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_1'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_2'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_3'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_4'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_5'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_6'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_7'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_8'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_9'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_10'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_11'-'voxel_extract', 'X_train', 'y_train', 'Lasso')
('X_train', 'voxel_extract_12'-'voxel_extract', 'X_train', 'y_

In [8]:
# evaluate fails to match y_tests key ('X_test', 'y_test', 'voxel_extract_i') with preds key 
# (('X_train', 'y_train', 'voxel_extract_i', 'Lasso'), ('X_test', 'y_test', 'voxel_extract_i'))
hard_metrics_set = Vset(name='hard_metrics', modules=[r2_score, explained_variance_score],
                             module_keys=["R2", "EV"])
hard_metrics = hard_metrics_set.evaluate(y_trains, preds)

for k, v in hard_metrics.items():
    if k != PREV_KEY:
        print(f'{k}: {v:.3f}')

('y_train', 'voxel_extract_0'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.171
('y_train', 'voxel_extract_1'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.216
('y_train', 'voxel_extract_2'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.148
('y_train', 'voxel_extract_3'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.167
('y_train', 'voxel_extract_4'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.162
('y_train', 'voxel_extract_5'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.191
('y_train', 'voxel_extract_6'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.213
('y_train', 'voxel_extract_7'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.215
('y_train', 'voxel_extract_8'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'Lasso', 'R2'): 0.235
('y_train', 'voxel_extract_9'-'voxel_extract', 'X_train', 'X_train', 'y_train', 'L